# Capstone Project - San Francisco Housing Sales Price 
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

In this project, I'll analyze each neighborhood in San Francisco and try to find a relationship between **San Francisco Housing Sales Price** and **nearby venues**. 

Specifically, this report will be targeted to stakeholders interested in investing a **real estate in San Francisco**, helping them to choose the regions with their favorite venues or lower real estate costs.

I'll create a map with each neighborhood in San Francisco segmented and clustered according to housing sales prices and nearby venues.

## Data <a name="data"></a>

In consider of our problem, I found following data sets:
* I found **Median Value Per Squre foot** data of each neighborhood in San Francisco, from Apr 1996 to Jul 2019. The csv file contains the **Region Name, City, CountyName, SizeRank and median_value_per_sqft** during this time period in USD.
* I used **Forsquare API** to get the most common venues of given neighborhood of San Francisco.
* I used **Google Maps API geocoding** to get the center cooridnates of each neighborhood.

## Methodology <a name="methodology"></a>

## Analysis <a name="analysis"></a>

## Results and Discussion <a name="results"></a>

## Conclusion <a name="conclusion"></a>